In [0]:
dbutils.widgets.text("catalog", "usa", "catalog")
dbutils.widgets.text("schema", "movies", "schema")

In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

In [0]:
mongodb_username = dbutils.secrets.get(scope="movies", key="mongodb_username")
mongodb_password = dbutils.secrets.get(scope="movies", key="mongodb_password")
mongodb_uri = dbutils.secrets.get(scope="movies", key="mongodb_uri")

In [0]:
uri = "mongodb+srv://{}:{}@{}/".format(mongodb_username, mongodb_password, mongodb_uri)
database = "sample_mflix"
collection = "movies"


In [0]:
df = spark.read.format("mongodb") \
  .option("spark.mongodb.read.connection.uri", uri) \
  .option("database", database) \
  .option("collection", collection) \
  .load()

In [0]:
df.printSchema()

In [0]:
display(df)

In [0]:
from pyspark.sql.functions import lit, current_timestamp

df2 = (
    df.withColumn("operationType", lit("insert"))
      .withColumn("coll", lit("samples_mflix"))
      .withColumn("db", lit("movies"))
      .withColumn("wallTime", current_timestamp())
)
display(df2)

In [0]:
df2.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable("{}.{}.bronze_mongodb_movies_backfill".format(catalog, schema))